# Beskrivelse
Her gives det det samme kode som i Eksemple_CNN.ipynb. Modelen der kunne når et test accuracy på 95%. Her skal vi undersøg om vi kan lave modellen bedre, eller dårligere, og se hvordan at ændre de forskellige parametere har effekt på resultatet.

Først importerer vi pakker og overfører datasætet.

In [1]:
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
from torch.utils.data import DataLoader, random_split
from CNN_utils.train import train
from CNN_utils.test import test

# Denne transform funktion gives til datasættet for at billederne kommer ud i den rigtig format, som er matricer med værdier mellem 0 og 1.
# Vi normalisere pixlerne fra [0, 255] til [0, 1] fordi mest ML algoritmer er bygget til at arbejde bedst med normaliseret data.
def image_transform(img):
    return torchvision.transforms.ToTensor()(img).unsqueeze(0)

# Overfører CIFAR10 træning og test datasætene fra pytorch
train_set = torchvision.datasets.MNIST(root='./data', train=True, download=True, transform=image_transform)
test_set = torchvision.datasets.MNIST(root='./data', train=False, download=True, transform=image_transform)

# Splitter træning sætet til en træning og validering sæt.
# val_set_ratio bestemmer hvor meget af sættet bliver brugt til validering.
val_set_ratio = 0.1
train_set, val_set = random_split(train_set, [int(len(train_set)*(1-val_set_ratio)), int(len(train_set)*val_set_ratio)])

# Tjekker størrelsen af billederne og hvor mange klasser der er
print("Images shape:", train_set[0][0].shape)
print("Number of classes:", len(np.unique(test_set.targets)))

# Model / data parameter
num_classes = 10
input_shape = (1, 28, 28)

# Laver dataloadere der samler vores data i batches og shuffler dem hvis vi vil gerne
BATCH_SIZE = 64
train_loader = DataLoader(train_set, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_set, batch_size=BATCH_SIZE, shuffle=True)
test_loader = DataLoader(test_set, batch_size=BATCH_SIZE, shuffle=False)

Images shape: torch.Size([1, 1, 28, 28])
Number of classes: 10


Køre den initial model igen så du kan sammenlign med den senere:

In [ ]:
class Net(nn.Module):
    """
    Netværksarkitektur for klassifikation af billeder
    
    Args:
    nn.Module: Superklasse for alle neurale netværk i PyTorch
    
    Returns:
    Net: Netværksarkitektur
    """
    def __init__(self, hyperparameters: dict = {}, input_shape = (1, 28, 28), num_classes: int = 3):
        # Initialiserer architecturen
        super(Net, self).__init__()
        self.input_shape = input_shape
        self.conv1 = nn.Conv2d(input_shape[0], 32, 3, 1)
        self.conv2 = nn.Conv2d(32, 64, 3, 1)
        self.dropout1 = nn.Dropout(0.25)
        self.dropout2 = nn.Dropout(0.5)
        # Input størrelsen er følgende: (Antal output kanaler af conv2*((billedets højde minus 4 på grund af de 2 convolutions)/2 på grund af maxpool2D)*Det samme men med billedets bredde)
        self.fc1 = nn.Linear(64*((self.input_shape[1]-4)//2)*((self.input_shape[2]-4)//2), 128)
        self.fc2 = nn.Linear(128, num_classes)  

        # Vælg loss function og optimizer
        self.criterion = nn.CrossEntropyLoss
        self.optimizer = optim.SGD
        self.hyperparameters = hyperparameters

    def forward(self, x: torch.Tensor):
        """
        Forward pass af netværket
        
        Args:
        x (torch.Tensor): Input tensor
        
        Returns:
        torch.Tensor: Output tensor
        """
        x = x.reshape([-1] + list(self.input_shape))
        x = self.conv1(x)
        x = F.relu(x)
        x = self.conv2(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 2)
        x = self.dropout1(x)
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.dropout2(x)
        x = self.fc2(x)
        output = F.log_softmax(x, dim=1)
        return output
    
    def predict(self, x: np.ndarray):
        """
        Forudsig klasse
        
        Args:
        x (np.ndarray): Input data
        
        Returns:
        torch.Tensor: Forudsiget klasse
        """
        # Konverter til tensor
        x = torch.Tensor(x).float()
        x = x.reshape([-1] + list(self.input_shape))

        # Forudsig klasse
        y_hat_prob = self(x)
        y_hat = torch.argmax(y_hat_prob, dim=1)

        return y_hat.detach().numpy()[0], y_hat_prob[0].detach().numpy()

model = Net({"epochs": 15}, input_shape=input_shape, num_classes=num_classes)

train(train_loader, val_loader, model)

## Opgaver: Undersøg forskellige modeller
### Convolution lagene
Nu skal du ændre parameter i modellen og se hvis de har en effekt på performance (val_accuracy) og træning hastighed (ms/step). Første prøv at ændre antal af convolution lag. Fjern den første convolution lag og se hvad der sker med validering accuracy:

In [ ]:
class Net(nn.Module):
    """
    Netværksarkitektur for klassifikation af billeder
    
    Args:
    nn.Module: Superklasse for alle neurale netværk i PyTorch
    
    Returns:
    Net: Netværksarkitektur
    """
    def __init__(self, hyperparameters: dict = {}, input_shape = (1, 28, 28), num_classes: int = 3):
        # Initialiserer architecturen
        super(Net, self).__init__()
        self.input_shape = input_shape
        self.conv1 = nn.Conv2d(input_shape[0], 32, 3, 1)
        self.conv2 = nn.Conv2d(32, 64, 3, 1)
        self.dropout1 = nn.Dropout(0.25)
        self.dropout2 = nn.Dropout(0.5)
        # Input størrelsen er følgende: (Antal output kanaler af conv2*((billedets højde minus 4 på grund af de 2 convolutions)/2 på grund af maxpool2D)*Det samme men med billedets bredde)
        self.fc1 = nn.Linear(64*((self.input_shape[1]-4)//2)*((self.input_shape[2]-4)//2), 128)
        self.fc2 = nn.Linear(128, num_classes)  

        # Vælg loss function og optimizer
        self.criterion = nn.CrossEntropyLoss
        self.optimizer = optim.SGD
        self.hyperparameters = hyperparameters

    def forward(self, x: torch.Tensor):
        """
        Forward pass af netværket
        
        Args:
        x (torch.Tensor): Input tensor
        
        Returns:
        torch.Tensor: Output tensor
        """
        x = x.reshape([-1] + list(self.input_shape))
        x = self.conv1(x)
        x = F.relu(x)
        x = self.conv2(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 2)
        x = self.dropout1(x)
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.dropout2(x)
        x = self.fc2(x)
        output = F.log_softmax(x, dim=1)
        return output
    
    def predict(self, x: np.ndarray):
        """
        Forudsig klasse
        
        Args:
        x (np.ndarray): Input data
        
        Returns:
        torch.Tensor: Forudsiget klasse
        """
        # Konverter til tensor
        x = torch.Tensor(x).float()
        x = x.reshape([-1] + list(self.input_shape))

        # Forudsig klasse
        y_hat_prob = self(x)
        y_hat = torch.argmax(y_hat_prob, dim=1)

        return y_hat.detach().numpy()[0], y_hat_prob[0].detach().numpy()

model = Net({"epochs": 15}, input_shape=input_shape, num_classes=num_classes)

train(train_loader, val_loader, model)

Nu prøve at fjerne den anden convolution lag istedet:

In [ ]:
class Net(nn.Module):
    """
    Netværksarkitektur for klassifikation af billeder
    
    Args:
    nn.Module: Superklasse for alle neurale netværk i PyTorch
    
    Returns:
    Net: Netværksarkitektur
    """
    def __init__(self, hyperparameters: dict = {}, input_shape = (1, 28, 28), num_classes: int = 3):
        # Initialiserer architecturen
        super(Net, self).__init__()
        self.input_shape = input_shape
        self.conv1 = nn.Conv2d(input_shape[0], 32, 3, 1)
        self.conv2 = nn.Conv2d(32, 64, 3, 1)
        self.dropout1 = nn.Dropout(0.25)
        self.dropout2 = nn.Dropout(0.5)
        # Input størrelsen er følgende: (Antal output kanaler af conv2*((billedets højde minus 4 på grund af de 2 convolutions)/2 på grund af maxpool2D)*Det samme men med billedets bredde)
        self.fc1 = nn.Linear(64*((self.input_shape[1]-4)//2)*((self.input_shape[2]-4)//2), 128)
        self.fc2 = nn.Linear(128, num_classes)  

        # Vælg loss function og optimizer
        self.criterion = nn.CrossEntropyLoss
        self.optimizer = optim.SGD
        self.hyperparameters = hyperparameters

    def forward(self, x: torch.Tensor):
        """
        Forward pass af netværket
        
        Args:
        x (torch.Tensor): Input tensor
        
        Returns:
        torch.Tensor: Output tensor
        """
        x = x.reshape([-1] + list(self.input_shape))
        x = self.conv1(x)
        x = F.relu(x)
        x = self.conv2(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 2)
        x = self.dropout1(x)
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.dropout2(x)
        x = self.fc2(x)
        output = F.log_softmax(x, dim=1)
        return output
    
    def predict(self, x: np.ndarray):
        """
        Forudsig klasse
        
        Args:
        x (np.ndarray): Input data
        
        Returns:
        torch.Tensor: Forudsiget klasse
        """
        # Konverter til tensor
        x = torch.Tensor(x).float()
        x = x.reshape([-1] + list(self.input_shape))

        # Forudsig klasse
        y_hat_prob = self(x)
        y_hat = torch.argmax(y_hat_prob, dim=1)

        return y_hat.detach().numpy()[0], y_hat_prob[0].detach().numpy()

model = Net({"epochs": 15}, input_shape=input_shape, num_classes=num_classes)

train(train_loader, val_loader, model)

Hvad hvis du sætter et ekstra convolution lag. Initialisere en ny Conv2D og sæt den efter MaxPooling2D. Du kan selv vælge parametrene af laget:

In [ ]:
class Net(nn.Module):
    """
    Netværksarkitektur for klassifikation af billeder
    
    Args:
    nn.Module: Superklasse for alle neurale netværk i PyTorch
    
    Returns:
    Net: Netværksarkitektur
    """
    def __init__(self, hyperparameters: dict = {}, input_shape = (1, 28, 28), num_classes: int = 3):
        # Initialiserer architecturen
        super(Net, self).__init__()
        self.input_shape = input_shape
        self.conv1 = nn.Conv2d(input_shape[0], 32, 3, 1)
        self.conv2 = nn.Conv2d(32, 64, 3, 1)
        self.dropout1 = nn.Dropout(0.25)
        self.dropout2 = nn.Dropout(0.5)
        # Input størrelsen er følgende: (Antal output kanaler af conv2*((billedets højde minus 4 på grund af de 2 convolutions)/2 på grund af maxpool2D)*Det samme men med billedets bredde)
        self.fc1 = nn.Linear(64*((self.input_shape[1]-4)//2)*((self.input_shape[2]-4)//2), 128)
        self.fc2 = nn.Linear(128, num_classes)  

        # Vælg loss function og optimizer
        self.criterion = nn.CrossEntropyLoss
        self.optimizer = optim.SGD
        self.hyperparameters = hyperparameters

    def forward(self, x: torch.Tensor):
        """
        Forward pass af netværket
        
        Args:
        x (torch.Tensor): Input tensor
        
        Returns:
        torch.Tensor: Output tensor
        """
        x = x.reshape([-1] + list(self.input_shape))
        x = self.conv1(x)
        x = F.relu(x)
        x = self.conv2(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 2)
        x = self.dropout1(x)
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.dropout2(x)
        x = self.fc2(x)
        output = F.log_softmax(x, dim=1)
        return output
    
    def predict(self, x: np.ndarray):
        """
        Forudsig klasse
        
        Args:
        x (np.ndarray): Input data
        
        Returns:
        torch.Tensor: Forudsiget klasse
        """
        # Konverter til tensor
        x = torch.Tensor(x).float()
        x = x.reshape([-1] + list(self.input_shape))

        # Forudsig klasse
        y_hat_prob = self(x)
        y_hat = torch.argmax(y_hat_prob, dim=1)

        return y_hat.detach().numpy()[0], y_hat_prob[0].detach().numpy()

model = Net({"epochs": 15}, input_shape=input_shape, num_classes=num_classes)

train(train_loader, val_loader, model)

Sammenlign hastighed og accuracy forskel mellem de tre sidste modeller og den basal model. Var de bedre eller dårligere? Hurtigere eller langsomere? Hvorfor tror du det?

### Max pooling lagene
Nu kigger vi på max pooling. Først prøv at fjern alle max pooling lagene og se hvad der sker med hastighed og accuracy:

In [ ]:
model = keras.Sequential(
    [
        keras.Input(shape=input_shape),
        layers.Conv2D(32, kernel_size=(3, 3), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Conv2D(64, kernel_size=(3, 3), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Flatten(),
        layers.Dense(num_classes, activation="softmax"),
    ]
)

model.summary()

batch_size = 128
epochs = 5

model.compile(loss="categorical_crossentropy", optimizer="SGD", metrics=["accuracy"])

history = model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, validation_split=0.1)

Nu prøve at set et ekstre max pooling lag før den først convolution lag:

In [ ]:
model = keras.Sequential(
    [
        keras.Input(shape=input_shape),
        layers.Conv2D(32, kernel_size=(3, 3), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Conv2D(64, kernel_size=(3, 3), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Flatten(),
        layers.Dense(num_classes, activation="softmax"),
    ]
)

model.summary()

batch_size = 128
epochs = 5

model.compile(loss="categorical_crossentropy", optimizer="SGD", metrics=["accuracy"])

history = model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, validation_split=0.1)

Hvad hvis vi bruger større kerner? Prøv kerne størrelse på (4,4) istedet for (2,2) i de to max pooling lage. Sådan at billederne bliver mindre:

In [ ]:
model = keras.Sequential(
    [
        keras.Input(shape=input_shape),
        layers.Conv2D(32, kernel_size=(3, 3), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Conv2D(64, kernel_size=(3, 3), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Flatten(),
        layers.Dense(num_classes, activation="softmax"),
    ]
)

model.summary()

batch_size = 128
epochs = 5

model.compile(loss="categorical_crossentropy", optimizer="SGD", metrics=["accuracy"])

history = model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, validation_split=0.1)

Sammenlign de sidste tre modeler der bruger forskellige max pooling lag med den initial model. Hvorfor tror du der er forskel i træning hastighed og accuracy?

### Aktivering funktionet
Aktivering funktionerne er brugt til at gør sikkert at modelen bliver ikke lineart, da en model uden aktivering kan kun finde lineart afhængigheder mellem dataen. Man kan alligevel skift aktivering funktion til en linear funktion ved at bruge strengen "linear" istedet for "relu". Træn en model der bruger kun linear aktivering i dens convolution lage:

In [ ]:
model = keras.Sequential(
    [
        keras.Input(shape=input_shape),
        layers.Conv2D(32, kernel_size=(3, 3), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Conv2D(64, kernel_size=(3, 3), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Flatten(),
        layers.Dense(num_classes, activation="softmax"),
    ]
)

model.summary()

batch_size = 128
epochs = 5

model.compile(loss="categorical_crossentropy", optimizer="SGD", metrics=["accuracy"])

history = model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, validation_split=0.1)

Der findes også andre ikke lineart funktioner, prøv sigmoid og tanh aktivering funktionerne:

In [ ]:
model = keras.Sequential(
    [
        keras.Input(shape=input_shape),
        layers.Conv2D(32, kernel_size=(3, 3), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Conv2D(64, kernel_size=(3, 3), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Flatten(),
        layers.Dense(num_classes, activation="softmax"),
    ]
)

model.summary()

batch_size = 128
epochs = 5

model.compile(loss="categorical_crossentropy", optimizer="SGD", metrics=["accuracy"])

history = model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, validation_split=0.1)

In [ ]:
model = keras.Sequential(
    [
        keras.Input(shape=input_shape),
        layers.Conv2D(32, kernel_size=(3, 3), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Conv2D(64, kernel_size=(3, 3), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Flatten(),
        layers.Dense(num_classes, activation="softmax"),
    ]
)

model.summary()

batch_size = 128
epochs = 5

model.compile(loss="categorical_crossentropy", optimizer="SGD", metrics=["accuracy"])

history = model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, validation_split=0.1)

Sammenlign de forskellige aktiveringsfunktioner. Skal man altid bruge et non-lineart aktiveringsfunktion? Får man hurtigere træning ved at bruge lineart aktivering istedet for ikke lineart aktivering? Er de andre aktivering funktioner så god som RELU?

## Opgave: Undersøg træning parameter
Det er ikke kun modellen der er vigtig for at få god resultater. Selve træning processen er også vigtigt.

Epochen bestemer hvor langt modellen skal træn, for mange epocher kan resulterer i overfitting, og for færre epocher resulterer i underfitting.

Batch størrelsen bestemmer hvor mange data punkter man bruge til at adjusterer vægtene med og hvor mange steps man tager per epoch som bestemmer hvor hurtigt vægtene ændres. Større batches resulterer i mere præcise vægt ændringer, men kan også resultarer i langsommere træning.

Der også findes mange forskellige optimering algoritmer der opdaterer vægtene.

### Antal Epocher
Nu skal du undersøg om de førrig modeller underfitter. Prøv at øve antal epocher og se hvordan accuracy ændres. Pas på med at gå alt for stort, da flere epocher tager mere tid til at træne prøv at ikke gå over 20:

In [ ]:
model = keras.Sequential(
    [
        keras.Input(shape=input_shape),
        layers.Conv2D(32, kernel_size=(3, 3), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Conv2D(64, kernel_size=(3, 3), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Flatten(),
        layers.Dense(num_classes, activation="softmax"),
    ]
)

model.summary()

batch_size = 128
epochs = 5

model.compile(loss="categorical_crossentropy", optimizer="SGD", metrics=["accuracy"])

history = model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, validation_split=0.1)

Hint: Et måde at finde en god epoch mængde er at starte med et stort epoch tal, og så se hvornår validering accuracy starter med at ændrer sig for lidt mellem hvert epoch (fx. 0.1). Dette strategi kaldes for early stopping, og den bruges for at gøre sikker at modellen træner ikke mere end den har bruge for.

Hvad fandt du til at være en god mængde af epocher? Giver det mening at bruge det ekstra tid til at for den mængde øvede accuracy man får? Hvad hvis din model tog 10 minutter eller en time per epoch? 

### Undersøg træning af vores model
Vi har gemt alt statistik fra vores træning under history variablen. Fra den kan vi hente udvikling af vores loss og accuracy igennem træning. Her skal du bruge matplotlib til at lave plotter til at vise udvikling af loss og accuracy. Prøve at se om modellen træner stabilt, eller om der er problemer som over og underfitting ved bruge af de her plotter.

In [ ]:
# Loss af træning og validering data er givet, lave en matplotlib plot der viser dem:
train_loss = history.history["loss"]
val_loss = history.history["val_loss"]



In [ ]:
# Accuracy af træning og validering data er givet, lave en matplotlib plot der viser dem:
train_accuracy = history.history["accuracy"]
val_accuracy = history.history["val_accuracy"]



### Andre optimering algoritmer
Lige nu bruges der SGD som optimering algoritm. Der andre optimizers i keras som kan ses [her](https://keras.io/api/optimizers/) prøv at bruge adam istedet for SGD, som er nyere og er den mest brugt inden for ML lige nu.

In [ ]:
model = keras.Sequential(
    [
        keras.Input(shape=input_shape),
        layers.Conv2D(32, kernel_size=(3, 3), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Conv2D(64, kernel_size=(3, 3), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Flatten(),
        layers.Dense(num_classes, activation="softmax"),
    ]
)

model.summary()

batch_size = 128
epochs = 20

model.compile(loss="categorical_crossentropy", optimizer="SGD", metrics=["accuracy"])

history = model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, validation_split=0.1)

Var Adam bedre end SGD? Har Adam så mange Epochs som SGD? Brug din plot kode blokke igen for at se om Adam overfitter datasættet:

In [ ]:
# Loss af træning og validering data er givet, lave en matplotlib plot der viser dem:
train_loss = history.history["loss"]
val_loss = history.history["val_loss"]



In [ ]:
# Accuracy af træning og validering data er givet, lave en matplotlib plot der viser dem:
train_accuracy = history.history["accuracy"]
val_accuracy = history.history["val_accuracy"]



## Opgave: En Sidste Model
Nu har du prøvet mange forskellige modeller. Prøv at træne en model med de parametere du tror vil virke bedste og træn det. Når du tror at du har trænet den bedste model, så køre test kodeblokken og se hvor god din model kan detekterer hånd skrevet nummer:

In [ ]:
model = keras.Sequential(
    [
        keras.Input(shape=input_shape),
        layers.Conv2D(32, kernel_size=(3, 3), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Conv2D(64, kernel_size=(3, 3), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Flatten(),
        layers.Dense(num_classes, activation="softmax"),
    ]
)

model.summary()

batch_size = 128
epochs = 5

model.compile(loss="categorical_crossentropy", optimizer="SGD", metrics=["accuracy"])

history = model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, validation_split=0.1)

## VENT!
Husk at du skal kun bruge test datasætet en gang til sidste. Er du sikker at din nuværende model er den du vil gerne teste på?

In [ ]:
score = model.evaluate(x_test, y_test, verbose=0)
print("Test loss:", score[0])
print("Test accuracy:", score[1])

Hvor meget bedre er denne model end den initial model i Eksemple_CNN.ipynb?